<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/ciclo2/Copia_de_3_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1AQr9H9bXDeNPchTRufU78g8z0yxHvrmC" width="100%">

# Preprocesamiento
---

En este _notebook_ mostraremos un ejemplo práctico de preprocesamiento de textos usando un _corpus_ en español de _Kaggle_ llamado [Spanish tweets suggesting depression](https://www.kaggle.com/datasets/francescoronzano/spanish-tweets-suggesting-depression).

<img src="https://drive.google.com/uc?export=view&id=1Vh514lihPERVhRK4pxuwu157IGSvFM7k" width="60%">

## **1. Carga de Datos**

En este caso disponemos de un conjunto de datos tabular (formato `csv`) que contiene 4 columnas:

- `id`: identificador único de cada tweet.
- `user_id`: identificador único de un usuario.
- `text`: texto del tweet.
- `date`: fecha de creación del tweet.

<img src="https://drive.google.com/uc?export=view&id=1mtV9eevIWZcJHq9aWRBHkn5kIr8pHBvf" width="60%">

Vamos a cargar el conjunto de datos con `pandas`. Primero importamos la librería:

In [ ]:
import pandas as pd
from IPython.display import display

Ahora, cargamos el conjunto de datos en la variable `df`:

In [ ]:
df = pd.read_csv(
        "https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u2/tweets_spa.csv",
        index_col=0
    )
display(df.head())

Validemos los tipos de las columnas que cargamos:

In [ ]:
display(df.dtypes)

## **2. Estructura Típica de Preprocesamiento**
---

En este caso veremos algunas de las operaciones más comunes en distintos preprocesamientos de información textual. Es necesario resaltar que no en todos los casos necesitará aplicar todas las técnicas e incluso en alguna aplicación específica va a necesitar estrategias más especializadas, no obstante, lo que presentamos en este caso aplica de forma general a la gran mayoría de aplicaciones de Procesamiento de Lenguaje Natural.

Veamos un diagrama de los **componentes de preprocesamiento** que veremos en este _notebook_:

<img src="https://drive.google.com/uc?export=view&id=1LZ21z7HWloUlrZPxQPgG2656VPXgglUl" width="100%">

Para estos ejemplos diseñaremos el preprocesamiento para un único documento y, posteriormente, aplicaremos todo el **preprocesamiento** al _corpus_ al finalizar el _notebook_:

In [ ]:
text = df['text'].iloc[2]
print(text)

## **3. Tokenizado del Texto**
---

El proceso de tokenizado consiste en separar el texto en unidades lógicas (caracteres, palabras, oraciones).

Para esto usaremos `spacy`, comenzamos importándolo:

In [ ]:
import spacy

Descargamos el _Pipeline_ para el español:

In [ ]:
spacy.cli.download("es_core_news_sm")

Creamos el _Pipeline_:

In [ ]:
nlp = spacy.load("es_core_news_sm")
print(nlp)

Vamos a crear un documento de `spacy` a partir del texto:

In [ ]:
doc = nlp(text)
print(doc)

Podemos extraer los tokens a nivel de palabras del documento:

In [ ]:
tokens = [token for token in doc]
print(tokens)

Veamos el tipo de un elemento de `tokens` para validar que sea un `Token` de `spacy`:

In [ ]:
print(type(tokens[0]))

## **4. Filtrado de Palabras**
---

Una práctica común en el preprocesamiento de textos es el filtrado de _tokens_ según distintas condiciones.

<img src="https://drive.google.com/uc?export=view&id=1rgVQ933f6qXPslXbRLYE57MwvVl2b5hl" width="100%">

Por ejemplo, podemos eliminar todas las palabras que sean _stopwords_, para ello definimos una condición:

In [ ]:
condition = lambda token: not token.is_stop

Filtramos los tokens:

In [ ]:
filtered_tokens = list(filter(condition, tokens))
print(filtered_tokens)

Como se puede observar, eliminamos _tokens_ como `"a"`, `"las"`, `"y"`, `"los"`, entre otras que son muy comunes (y poco informativas) en Español.

También es posible filtrar palabras por longitud, por ejemplo, en español la palabra más larga tiene 23 letras (electroencefalografista), por ello, podemos definir una condición para filtrar palabras que tengan una longitud en un rango dado:

In [ ]:
condition = lambda token: len(token) > 0 and len(token) < 24

Filtramos los tokens:

In [ ]:
filtered_tokens2 = list(filter(condition, filtered_tokens))
print(filtered_tokens2)

## **5. Lematización**
---

En algunas aplicaciones no es de importancia la conjugación de las palabras (tiempo, plurales, géneros, ...), por lo cual, se suele transformar el texto a sus versiones lematizadas.

Vamos a aplicar este enfoque sobre cada uno de los _tokens_ filtrados:

In [ ]:
lemmas = [token.lemma_ for token in filtered_tokens2]
print(lemmas)

Como puede ver, palabras como `"trae"` se convierten en su infinitivo `"traer"`.

Finalmente, unimos todos los _tokens_ en un único _string_:

In [ ]:
lemma_text = " ".join(lemmas)
print(lemma_text)

## **6. Normalización de Caracteres**
---

En muchos idiomas tenemos caracteres modificadores de vocales o letras. Por ejemplo, en español tenemos las tildes y la letra "ñ"; o en el caso del portugués tenemos tres tipos de acentos.

Este tipo de variaciones pueden ser eliminadas en un proceso de normalización de los textos con distintos tipos de codificaciones. Para el español es muy común usar la librería `unidecode`, vamos a instalarla:

In [ ]:
!pip install unidecode

Ahora podemos importarla:

In [ ]:
from unidecode import unidecode

Veamos cómo queda el texto luego de normalizarlo:

In [ ]:
norm_text = unidecode(lemma_text)
print(norm_text)

Como puede ver, ya no hay tildes en el texto.

## **7. Modificación de la Grafía**
---

Normalmente, los textos se suelen procesar en minúsculas. Con esto eliminamos modificadores relacionados con el inicio de un texto, palabras capitalizadas luego de signos de puntuación, entre otras cosas.

En este caso, convertimos el texto a minúsculas:

In [ ]:
lower_text = norm_text.lower()
print(lower_text)

## **8. Limpieza con Regex**
---

Comúnmente se suelen aplicar expresiones regulares para eliminar caracteres o secuencias de caracteres no deseadas. En el ejemplo que estamos desarrollando, podemos aplicar una expresión regular para eliminar todos los caracteres que no sean espacios ni letras minúsculas.

Primero importamos la librería para expresiones regulares:

In [ ]:
import re

Ahora, definimos una expresión regular que cumpla con los criterios mencionados:

In [ ]:
pat = re.compile(r"[^a-z ]")

Reemplazamos las coincidencias:

In [ ]:
clean_text = re.sub(pat, "", lower_text)
print(clean_text)

Como se puede ver, ahora tenemos espacios repetidos (ya que algunos _tokens_ se eliminaron), podemos usar una segunda expresión regular para eliminar espacios repetidos:

In [ ]:
spaces = re.compile(r"\s{2,}")

Reemplazamos espacios repetidos por un único espacio:

In [ ]:
spaces_text = re.sub(spaces, " ", clean_text)
print(spaces_text)

## **9. Preprocesamiento Completo**
---

Como pudimos ver en el paso anterior, tenemos un texto más limpio que puede llegar a ser más fácil de analizar de manera automática.

A continuación veremos la aplicación del preprocesamiento sobre el _corpus_ completo. Para ello, definimos la función `preprocess` la cual toma como entrada un documento cualquiera y retorna un documento preprocesado:

In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")

def preprocess(text, min_len=1, max_len=23):
    # Creamos documento de spacy
    doc = nlp(text)
    # Eliminamos stopwords
    filtered_tokens = filter(
            lambda token: not token.is_stop,
            doc
            )
    # Filtramos palabras por longitud
    filtered_tokens2 = filter(
            lambda token: len(token) >= min_len and len(token) <= max_len,
            filtered_tokens
        )
    # Obtenemos los lemmas de cada token
    lemmas = map(
            lambda token: token.lemma_,
            filtered_tokens2
            )
    lemma_text = " ".join(lemmas)
    # Normalizamos el texto
    norm_text = unidecode(lemma_text)
    # Quitamos grafía
    lower_text = norm_text.lower()
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", lower_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

Veamos un ejemplo de la función:

In [ ]:
prep_text = preprocess(df['text'].iloc[0])
print(prep_text)

Podemos usar el método `apply` de los dataframes de `pandas` para aplicar el preprocesamiento sobre todo el _corpus_:

In [ ]:
prep_corpus = df['text'].apply(preprocess).tolist()
print(prep_corpus[:10])

Como se puede ver, obtuvimos un _corpus_ más estandarizado y compacto con un flujo típico de preprocesamiento.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Text Preprocessing in Natural Language Processing](https://towardsdatascience.com/text-preprocessing-in-natural-language-processing-using-python-6113ff5decd8).
- [Natural Language Processing | Text Preprocessing | Spacy vs NLTK](https://medium.com/nerd-for-tech/natural-language-processing-text-preprocessing-spacy-vs-nltk-b70b734f5560).
- _Fuente de los íconos_
    - Flaticon. Twitter free icon [PNG]. https://www.flaticon.com/free-icon/twitter_2504947
    - Flaticon. User free icon [PNG]. https://www.flaticon.com/free-icon/user_3177440
    - Flaticon. Document free icon [PNG]. https://www.flaticon.com/free-icon/document_888071
    - Flaticon. Documents File free icon [PNG]. https://www.flaticon.com/free-icon/documents_3135874
    - Flaticon. Filter free icon [PNG]. https://www.flaticon.com/free-icon/filter_7783331
    - Flaticon. Transformation free icon [PNG]. https://www.flaticon.com/free-icon/transformation_1139032
    - Flaticon. Languages free icon [PNG]. https://www.flaticon.com/free-icon/languages_3898150
    - Flaticon. Letter A free icon [PNG]. https://www.flaticon.com/free-icon/letter-a_3665909
    - Flaticon. Letter a free icon [PNG]. https://www.flaticon.com/free-icon/letter-a_3665887
    - Flaticon. Down Right Arrow free icon [PNG]. https://www.flaticon.com/free-icon/down-right-arrow_3272625
    - Flaticon. Broom free icon [PNG]. https://www.flaticon.com/free-icon/broom_2954880
    - Flaticon. Hierarchy free icon [PNG]. https://www.flaticon.com/free-icon/hierarchy_4657126

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*